# Battleship API

I don't know how to work with mutable state in Haskell without desecrating the heap, so I decided to solve the problem in Python. First a few points:

- Below is the battleship state in a 4x4 grid using just the Submarine, Cruiser and Patrol. It is easy to make the code scale up by changing gridDimensionX, gridDimensionY and adding more boats to the dictionary shipLengths.
- I have supplied the code in a Jupyter notebook instead of a runnable python script. Since I'm not exactly sure what is going to be built on top, I decided that constructing a simple object with the correct methods would be more useful than trying to package it up into a script. I could be wrong though.......

The gamestate is stored as an object which records where each player has fired and where their boats are. There is also an error message which contains information about how an attack or boat placement went and an active player who is the current attacker / boat placer. 

I'm not sure what you were looking for exactly, but it was a fun little problem to solve, so thanks!

In [1]:
import random

In [2]:
#(0,0) is the top left corner. We index grip points like you would index a matrix.
gridDimensionX = 4
gridDimensionY = 4
def validGridPoint(i,j):
    """
    test if a grid point is valid
    """
    return (0 <= i < gridDimensionX) and (0 <= j < gridDimensionY)

In [3]:
def blankGrid(DimensionX,DimensionY,x):
    """
    produce an empty grid
    """
    return {(i,j) : x for i in range(DimensionX) for j in range(DimensionY)}

In [4]:
def computeShipCoordinates(i,j,direction,length):
    """
    we need to place ships on the board. 
    (i.j) is the coordinate of the base of the ship.
    direction is "North", "South", "East" or "West"
    length is the length of the ship.
    """
    if direction == "North":
        return [(i,j-c) for c in range(length)]
    elif direction == "East":
        return [(i+c,j) for c in range(length)]
    elif direction == "South":
        return [(i,j+c) for c in range(length)]
    elif direction == "West":
        return [(i-c,j) for c in range(length)]

In [5]:
class GameState:
    """
    players:  
    0 = player0
    1 = player1
    
    peg colors: 
    "R" = Red   (HIT)
    "W" = White (MISS)
    "E" = Empty
    
    
    errors: 
    "NoError"           = everything went OK
    "NotValidGridPoint" = someone tried to shoot outside the grid or place a ship outside the grid
    "PegAlreadyPlaced"  = someone tried to shoot at a location where they have already shot
    "ShipCollision"     = someone tried to place one ship ontop of another
    
    """
    def __init__(self):
        """
        self.shotsFired[i] is a directionary recording where player i shot and if they hit/missed an enemey ship
        self.boats[i] is a dictionary recording boat locations for player i
        self.activePlayer records whose turn it is
        """
        self.shotsFired = {0 : blankGrid(gridDimensionX,gridDimensionY,"E"),
                           1 : blankGrid(gridDimensionX,gridDimensionY,"E")}
        self.boats = {0 : [], 1 : []}
        self.error = "NoError"
        self.activePlayer = 0
        
    def otherPlayer(self):
        if self.activePlayer == 0:
            return 1
        else:
            return 0
    
    def changePlayer(self):
        self.activePlayer = self.otherPlayer()
        return self.activePlayer

    
    
    def attack(self,i,j):
        """
        for an attack to be valid, it must be directed at a valid grid point with no peg.
        """
        if not validGridPoint(i,j):
            self.error = "NotValidGridPoint"
            return self.error
        elif self.shotsFired[self.activePlayer][(i,j)] != "E":
            self.error = "PegAlreadyPlaced"
            return self.error
        elif (i,j) in self.boats[self.otherPlayer()]:
            self.shotsFired[self.activePlayer][(i,j)] = "R"
            self.error = "NoError"
            return self.error
        else:
            self.shotsFired[self.activePlayer][(i,j)] = "W"
            self.error = "NoError"
            return self.error
    
    
    
    def addShip(self,i,j,direction,length):
        """
        inorder to add a ship, it must be contained inside the grid and not intersect any other ships
        """
        ship = computeShipCoordinates(i,j,direction,length)
        for point in ship:
            if point in self.boats[self.activePlayer]:
                self.error = "ShipCollision"
                return self.error
            elif not validGridPoint(point[0],point[1]):
                self.error = "NotValidGridPoint"
                return self.error
        self.boats[self.activePlayer] = self.boats[self.activePlayer] + ship
        self.error = "NoError"
        return self.error
    
    
    def addRandomShip(self,length):
        """
        to add a random ship of a given length, we keep trying to add ships until one sticks
        """
        status = "NothingYet"
        while status != "NoError":
            i = random.choice(range(gridDimensionX))
            j = random.choice(range(gridDimensionY))
            direction = random.choice(["North", "South", "East" or "West"])
            status = self.addShip(i,j,direction,length)
        return self.error
    
    def activePlayerWin(self):
        for point in self.boats[self.activePlayer]:
            if self.shotsFired[self.activePlayer][point] != "R":
                return False
        return True

In [6]:
shipLengths = {"Submarine" : 3, "Cruiser" : 2, "Patrol" : 1}

def newgame():
    """
    create a new GameState with boats placed at random from shipLengths
    """
    gameState = GameState()
    for ship in shipLengths:
        gameState.addRandomShip(shipLengths[ship])
    
    
    gameState.changePlayer()
       
    for ship in shipLengths:
        gameState.addRandomShip(shipLengths[ship])
    
    
    gameState.changePlayer()
    return gameState

# Example Usage

In [7]:
test = newgame()

In [8]:
print(  test.activePlayer  )
print(  test.boats[0]  )
print(  test.boats[1]  )

0
[(0, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 3)]
[(1, 0), (2, 0), (2, 1), (2, 2), (0, 1), (0, 2)]


In [9]:
print(  test.attack(2,1)  )
print(  test.attack(2,2)  )
print(  test.attack(2,2)  )

NoError
NoError
PegAlreadyPlaced


In [10]:
test.shotsFired[0]

{(0, 0): 'E',
 (0, 1): 'E',
 (0, 2): 'E',
 (0, 3): 'E',
 (1, 0): 'E',
 (1, 1): 'E',
 (1, 2): 'E',
 (1, 3): 'E',
 (2, 0): 'E',
 (2, 1): 'R',
 (2, 2): 'R',
 (2, 3): 'E',
 (3, 0): 'E',
 (3, 1): 'E',
 (3, 2): 'E',
 (3, 3): 'E'}

In [11]:
test.changePlayer()

1

In [12]:
print(  test.attack(0,0)  )
print(  test.attack(0,1)  )
print(  test.attack(2,2)  )

NoError
NoError
NoError


In [13]:
test.shotsFired[1]

{(0, 0): 'W',
 (0, 1): 'W',
 (0, 2): 'E',
 (0, 3): 'E',
 (1, 0): 'E',
 (1, 1): 'E',
 (1, 2): 'E',
 (1, 3): 'E',
 (2, 0): 'E',
 (2, 1): 'E',
 (2, 2): 'R',
 (2, 3): 'E',
 (3, 0): 'E',
 (3, 1): 'E',
 (3, 2): 'E',
 (3, 3): 'E'}

In [14]:
print(  test.activePlayer  )

1
